# IDCCs traitées et non traitées pour les pages `contributions`

Dans cette exploration, le but est de récupérer pour chaque contribution générique, la liste des IDCCs sélectionnés

## 1. Chargement des librairies

In [ ]:
import pandas as pd
from src.elasticsearch_connector import ElasticsearchConnector

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)

## 2. Récupération des queries sur elasticsearch

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY_LOG_CONTRIB = {
    "query": {
        "bool": { 
          "must": [
            {
              "prefix": {
                "url": "https://code.travail.gouv.fr/contribution" 
              }
            },
            {
              "range": {
               "logfile": {
                    "gte": "2024-05-01",
                    "lte": "2024-08-01"
               }
              }
            }
          ]
        }
      }
}

In [ ]:
logs = es_connector.execute_query(QUERY_LOG_CONTRIB, "logs-new")

## 3. Vue d'ensemble

### Parcours 1 : Je connais ma convention collective

In [ ]:
# Récupération des logs des urls de contribution génériques
logs_generic = logs[~logs["url"].str.contains(r"contribution/\d{1,4}-", regex=True)]

logs_generic_cc_select_traitée_et_non_traitée = logs_generic[
    (logs_generic["type"] == "cc_select_non_traitée") | 
    (logs_generic["type"] == "cc_select_traitée")
]

logs_generic_cc_select_traitée_et_non_traitée["cleaned_url"] = logs_generic_cc_select_traitée_et_non_traitée["url"].str.split('#').str[0].str.split('?').str[0]

grouped = logs_generic_cc_select_traitée_et_non_traitée.groupby(['cleaned_url', 'idCc', 'type']).size().reset_index(name='count')

# Format the output
for url, group in grouped.groupby('cleaned_url'):
    print(f"url : {url}")
    for _, row in group.iterrows():
        cc = row['idCc']
        type = row['type']
        count = row['count']
        nb_visits = logs_generic_cc_select_traitée_et_non_traitée[logs_generic_cc_select_traitée_et_non_traitée["cleaned_url"] == url].shape[0]
        cc_select_traitée_total = logs_generic_cc_select_traitée_et_non_traitée[logs_generic_cc_select_traitée_et_non_traitée["type"] == "cc_select_traitée"].shape[0]
        cc_select_non_traitée_total = logs_generic_cc_select_traitée_et_non_traitée[logs_generic_cc_select_traitée_et_non_traitée["type"] == "cc_select_non_traitée"].shape[0]
        print(f"CC {cc} : {type} => {count} events on {nb_visits} visits")